<a href="https://colab.research.google.com/github/jrctechmind/machine_learning/blob/main/Modelagem_de_Sequ%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.decomposition import NMF

# Dados de exemplo: matriz de avaliações de usuários para itens e características dos itens
avaliacoes = np.array([
    [5, 3, 0, 0],
    [4, 0, 4, 1],
    [1, 1, 0, 5],
    [0, 0, 4, 4],
    [0, 1, 5, 4],
])

caracteristicas_itens = np.array([
    [1, 0, 1, 0],  # Características do item 0
    [1, 1, 0, 0],  # Características do item 1
    [0, 1, 0, 1],  # Características do item 2
    [0, 0, 1, 1]   # Características do item 3
])

def similaridade_usuario(usuario1, usuario2):
    """
    Calcula a similaridade entre dois usuários usando a correlação de Pearson.
    Quanto maior o valor retornado, mais similaridade entre os usuários.

    Argumentos:
    usuario1 -- índice do primeiro usuário
    usuario2 -- índice do segundo usuário

    Retorna:
    similaridade -- valor de similaridade entre os usuários
    """
    avaliacoes_usuario1 = avaliacoes[usuario1]
    avaliacoes_usuario2 = avaliacoes[usuario2]
    avaliacoes_comuns = np.logical_and(avaliacoes_usuario1 != 0, avaliacoes_usuario2 != 0)

    if np.sum(avaliacoes_comuns) == 0:
        return 0

    avaliacoes_usuario1_comuns = avaliacoes_usuario1[avaliacoes_comuns]
    avaliacoes_usuario2_comuns = avaliacoes_usuario2[avaliacoes_comuns]

    media_usuario1 = np.mean(avaliacoes_usuario1_comuns)
    media_usuario2 = np.mean(avaliacoes_usuario2_comuns)

    numerador = np.sum((avaliacoes_usuario1_comuns - media_usuario1) * (avaliacoes_usuario2_comuns - media_usuario2))
    denominador = np.sqrt(np.sum((avaliacoes_usuario1_comuns - media_usuario1)**2)) * np.sqrt(np.sum((avaliacoes_usuario2_comuns - media_usuario2)**2))

    if denominador == 0:
        return 0

    similaridade = numerador / denominador
    return similaridade

def recomendar(usuario_alvo, k=2, n_components=2, alpha=0.5):
    """
    Faz recomendações para um usuário usando um sistema de recomendação híbrido.

    Argumentos:
    usuario_alvo -- índice do usuário para o qual desejamos fazer recomendações
    k -- número de itens para recomendar (padrão: 2)
    n_components -- número de componentes latentes na fatoração de matriz (padrão: 2)
    alpha -- fator de ponderação para combinar as recomendações da filtragem colaborativa e baseada em conteúdo (padrão: 0.5)

    Retorna:
    recomendacoes -- lista de índices dos itens recomendados
    """
    # Filtragem colaborativa
    model = NMF(n_components=n_components, init='random', random_state=42)
    W = model.fit_transform(avaliacoes)
    H = model.components_
    recomendacoes_colaborativas = np.argsort(H[:, usuario_alvo])[::-1][:k]

    # Filtragem baseada em conteúdo
    avaliacoes_usuario = avaliacoes[usuario_alvo]
    itens_avaliados = np.where(avaliacoes_usuario != 0)[0]
    similaridades = [similaridade_usuario(item, itens_avaliados[0]) for item in range(len(avaliacoes[0]))]
    indices_similares = np.argsort(similaridades)[::-1][:k]
    recomendacoes_baseadas_em_conteudo = [item for item in indices_similares if item not in itens_avaliados]

    # Combinar recomendações
    recomendacoes = np.concatenate([recomendacoes_colaborativas, recomendacoes_baseadas_em_conteudo])
    recomendacoes_unicas, counts = np.unique(recomendacoes, return_counts=True)
    recomendacoes = recomendacoes_unicas[np.argsort(counts)][::-1][:k]

    return recomendacoes

# Exibição de instruções e interações com o usuário
print("Bem-vindo ao Sistema de Recomendação!")
print("Por favor, insira o número do usuário para o qual deseja fazer recomendações (0 a", len(avaliacoes) - 1, "):")
usuario_alvo = int(input())

if usuario_alvo < 0 or usuario_alvo >= len(avaliacoes):
    print("Número de usuário inválido. Por favor, insira um número dentro do intervalo válido.")
else:
    print("\nUsuário alvo selecionado:", usuario_alvo)
    print("Fazendo recomendações para o usuário...")
    recomendacoes = recomendar(usuario_alvo)
    print("\nRecomendações para o usuário selecionado:")
    for i, item in enumerate(recomendacoes):
        print("  - Item", item)

print("\nObrigado por usar nosso sistema de recomendação!")


Bem-vindo ao Sistema de Recomendação!
Por favor, insira o número do usuário para o qual deseja fazer recomendações (0 a 4 ):
1

Usuário alvo selecionado: 1
Fazendo recomendações para o usuário...

Recomendações para o usuário selecionado:
  - Item 1.0
  - Item 0.0

Obrigado por usar nosso sistema de recomendação!
